In [3]:
from asammdf import MDF, Signal
from pathlib import Path
import requests
from datetime import datetime, timedelta
from time import sleep

_path = Path('../files/decoded')
files = list(_path.rglob('*.[mM][fF]4'))
print(f'Found {len(files)} MF4 files in the current directory and subdirectories.')

Found 2 MF4 files in the current directory and subdirectories.


In [137]:
victoriametrics_url = "http://localhost:8428/api/v1/import/prometheus"

def get_channel_data(signal: Signal) -> tuple[str, int]:
    display_names = list(signal.display_names.keys())
    message = display_names[1].split('.')[0]
    can_id = display_names[2].split(' ')[0].split('ID=')[1]
    return message, int(can_id,16)

def make_metric_line(metric_name: str, message: str, can_id: int, unit: str, value: float, timestamp: datetime|float) -> str:
    # Format the metric line for Prometheus
    return f'{metric_name}{{message="{message}",can_id="{can_id:X}",unit="{unit}"}} {value} {timestamp.timestamp() if type(timestamp) is datetime else timestamp}\n'

def send_signal(signal: Signal, start_time: datetime):
    message, can_id = get_channel_data(signal)
    metric_name = signal.name.replace(' ', '_')
    unit = signal.unit if signal.unit else ''
    
    print(f'  => {metric_name} [{start_time} - {start_time + timedelta(seconds=signal.timestamps[-1])}]')
    for sample, ts in zip(signal.samples, signal.timestamps):
        data = make_metric_line(metric_name, message, can_id, unit, sample, start_time + timedelta(seconds=ts))
        requests.post(victoriametrics_url, data=data)        
    

In [49]:
mdf = MDF()

mdf1 = MDF(files[0])
mdf2 = MDF(files[1])

In [53]:
mdf1.info()

{'version': '4.10',
 'program': 'amdf8.5.',
 'comment': '<HDcomment>\n<TX/>\n<common_properties/>\n</HDcomment>',
 'groups': 117,
 'group 0': {'cycles': 1079,
  'comment': 'CAN1 ID=0x18EF1900 TMS_Cmd1 PGN=0xEF00 SA=0x0',
  'channels count': 8,
  'channel 0': 'name="time" type=MASTER',
  'channel 1': 'name="Cmd_Pump_Speed" type=VALUE',
  'channel 2': 'name="Cmd_Heater_power" type=VALUE',
  'channel 3': 'name="Cooling_Capacity" type=VALUE',
  'channel 4': 'name="Cmd_Battery_outlet_temp" type=VALUE',
  'channel 5': 'name="Cmd_Battery_inlet_temp" type=VALUE',
  'channel 6': 'name="Ambient_temp" type=VALUE',
  'channel 7': 'name="Cmd_Operating_Mode" type=VALUE'},
 'group 1': {'cycles': 1079,
  'comment': 'CAN1 ID=0x18EF8000 WP120_DC_EMP__MotorCommand PGN=0xEF00 SA=0x0',
  'channels count': 5,
  'channel 0': 'name="time" type=MASTER',
  'channel 1': 'name="DC_PercentRPMSpeed" type=VALUE',
  'channel 2': 'name="DC_MotorSpeed" type=VALUE',
  'channel 3': 'name="DC_PowerHold" type=VALUE',
  'ch

In [14]:
signals = list(mdf.iter_channels())
signal_names = [sig.name for sig in mdf.iter_channels()]

In [ ]:
def is_vm_busy():
    try:
        resp = requests.get("http://localhost:8428/api/v1/status/active_queries")
        if resp.status_code == 200:
            data = resp.json()
            # If there are active queries, consider VM busy
            return len(data.get('data', [])) > 0
        else:
            return False  # If can't check, assume not busy
    except Exception:
        return False

for sig in signals:
    # Wait until VictoriaMetrics is not busy
    while is_vm_busy():
        print("VictoriaMetrics is busy, waiting...")
        sleep(1)
    try:
        send_signal(sig, mdf.start_time)
        sleep(0.1)  # To avoid overwhelming the server with requests
    except Exception as e:
        print(f"Error sending signal {sig.name}: {e}")

  => Cmd_Pump_Speed [2025-06-21 15:01:08+00:00 - 2025-06-21 16:57:16.273850+00:00]
  => Cmd_Heater_power [2025-06-21 15:01:08+00:00 - 2025-06-21 16:57:16.273850+00:00]
  => Cooling_Capacity [2025-06-21 15:01:08+00:00 - 2025-06-21 16:57:16.273850+00:00]
  => Cmd_Battery_outlet_temp [2025-06-21 15:01:08+00:00 - 2025-06-21 16:57:16.273850+00:00]
  => Cmd_Battery_inlet_temp [2025-06-21 15:01:08+00:00 - 2025-06-21 16:57:16.273850+00:00]
  => Ambient_temp [2025-06-21 15:01:08+00:00 - 2025-06-21 16:57:16.273850+00:00]
  => Cmd_Operating_Mode [2025-06-21 15:01:08+00:00 - 2025-06-21 16:57:16.273850+00:00]
  => DC_PercentRPMSpeed [2025-06-21 15:01:08+00:00 - 2025-06-21 16:57:16.273850+00:00]
  => DC_MotorSpeed [2025-06-21 15:01:08+00:00 - 2025-06-21 16:57:16.273850+00:00]
  => DC_PowerHold [2025-06-21 15:01:08+00:00 - 2025-06-21 16:57:16.273850+00:00]
  => DC_OnOffDirection [2025-06-21 15:01:08+00:00 - 2025-06-21 16:57:16.273850+00:00]
  => BE_PercentRPMSpeed [2025-06-21 15:01:08+00:00 - 2025-06

In [10]:
mdf = MDF(files[0])

In [11]:
sig = mdf.select(['DCDC2_Set_index_byte'])[0]
len(sig.samples)

96392

In [12]:
sig = mdf.select(['DCDC2_Output_current_limit'])[0]
len(sig.samples)

69028

In [15]:
len(signals[0].samples)

1079